In [3]:
import pickle
import pandas as pd 
import os
import datasets
import re
from automatic_metrics import AutomaticMetrics
generated_path = "./generated"
dg_all_path: str = os.path.join(generated_path, "dg_all_strict/dg_all.pickle")

qg_path: str = os.path.join(generated_path, "qg_strict/qg_strict.pickle")

qag_path: str = os.path.join(generated_path, "qag_strict/correct/qag_strict.pickle")

dg_1_path: str = os.path.join(generated_path,"dg_1_strict/dg_1_multiple_base.pickle")

dg_1_path_combine = os.path.join(generated_path, "dg_1_strict/dg_1_multiple_combine.pickle")

with open(qg_path, "rb" ) as f :
    qg_val = pickle.load(f)

with open(qag_path, "rb") as f :
    qag_val = pickle.load(f)

with open(dg_all_path, "rb") as f:
    dg_all_val = pickle.load(f)

with open(dg_1_path, "rb") as f:
    dg_1_val = pickle.load(f) 

with open(dg_1_path_combine, "rb") as f:
    dg_1_val_combine = pickle.load(f)



In [4]:
qg_val_path = "VosLannack/squad_id_512"
dg_all_val_path = "VosLannack/race_id"
tyq_path = "google-research-datasets/tydiqa"

data_qg = datasets.load_dataset(qg_val_path)
data_dg = datasets.load_dataset(dg_all_val_path)
data_tyq = datasets.load_dataset(tyq_path, "secondary_task")

val_qg = pd.DataFrame(data_qg["validation"])
val_dg = pd.DataFrame(data_dg["validation"])
val_dg_1 = val_dg.copy()
val_dg_1_combine = val_dg_1.copy()

train_tyq = pd.DataFrame(data_tyq["train"])
val_tyq = pd.DataFrame(data_tyq["validation"])

In [3]:
val_qag = pd.DataFrame(data_qg["validation"])
val_qag.head()

,question,answer,context
0,Di negara apa Normandia berada?,Perancis,Bangsa Normandia (Norman: Nourmands; Prancis: ...
1,Kapan Normandia di Normandia?,Abad 10 dan 11,Bangsa Normandia (Norman: Nourmands; Prancis: ...
2,Dari negara mana asal Norse?,"Denmark, Islandia, dan Norwegia",Bangsa Normandia (Norman: Nourmands; Prancis: ...
3,Siapa pemimpin Norse?,Rollo,Bangsa Normandia (Norman: Nourmands; Prancis: ...
4,Abad berapa pertama kali Normandia mendapatkan...,Abad ke-10,Bangsa Normandia (Norman: Nourmands; Prancis: ...


In [4]:
val_tyq["id"].str.find("finnish") + 1

0       0
1       0
2       0
3       0
4       0
       ..
5072    0
5073    0
5074    0
5075    0
5076    0
Name: id, Length: 5077, dtype: int64

In [5]:
val_tyq["id"].str.find("indonesian") + 1 == 1

0       False
1       False
2       False
3       False
4       False
        ...  
5072    False
5073    False
5074    False
5075    False
5076    False
Name: id, Length: 5077, dtype: bool

In [6]:
val_tyq_indo = val_tyq.loc[val_tyq["id"].str.find("indonesian") + 1 == 1, :]
train_tyq_indo = train_tyq.loc[train_tyq["id"].str.find("indonesian") + 1 == 1, :]

In [22]:
val_dg_1.iloc[0,3]

'Timothy mempunyai beban yang berat. '

In [5]:
dg_1_val_combine[0]

['<pad> Timotius mempunyai harapan yang lebih baik.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad> Timotius mempunyai banyak waktu.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad> Timotius mempunyai harapan yang lebih rendah dibandingkan tepat waktu.</s>',
 '<pad> <unk> Timothy bersemangat dengan harga murah.</s><pad><pad><pad><pad> <unk> Timothy bersemangat dengan penampilannya yang buruk.</s><pad> <unk> Timothy bersemangat dengan penampilannya.</s><pad><pad><pad><pad><pad>',
 '<pad> Tidak ada yang mempunyai banyak waktu.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad> Tidak ada yang mempunyai banyak waktu luang dan menonton pertandingan selama begitu cepat.</s><pad> Tidak ada yang mempunyai banyak waktu luang.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>',
 '<pad> ketika mereka memiliki te

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def cosine_local_similarity(distractor, correct_answer):
    vectorizer = TfidfVectorizer()
    vectors = vectorizer.fit_transform([distractor, correct_answer])
    cosine_sim = cosine_similarity(vectors)[0, 1]
    return cosine_sim

# Example sentences
distractor = "Timothy memiliki beban yang berat."
correct_answer = "Timothy mempunyai beban yang berat."

cosine_diff = cosine_local_similarity(distractor, correct_answer)
print(f"Cosine similarity: {cosine_diff}")

Cosine similarity: 0.6694188517266485


In [18]:
def jaccard_similarity(distractor, correct_answer):
    # Convert sentences to sets of words
    set_distractor = set(distractor.split())
    set_correct_answer = set(correct_answer.split())
    
    # Calculate Jaccard similarity
    intersection = set_distractor.intersection(set_correct_answer)
    union = set_distractor.union(set_correct_answer)
    
    return len(intersection) / len(union) if union else 0

# Example sentences
distractor = "timothy memiliki beban yang berat."
correct_answer = "timothy mempunyai beban yang berat."

jaccard_sim = jaccard_similarity(distractor, correct_answer)
print(f"Jaccard Similarity: {jaccard_sim}")


Jaccard Similarity: 0.6666666666666666


In [27]:
import Levenshtein

def levenshtein_dissimilarity(distractor, correct_answer):
    distance = Levenshtein.distance(distractor, correct_answer)
    max_length = max(len(distractor), len(correct_answer))
    return distance / max_length

# Example sentences
distractor = "Timothy memiliki beban yang berat."
correct_answer = "Timothy mempunyai beban yang berat."

levenshtein_diff = levenshtein_dissimilarity(distractor, correct_answer)
print(f"Levenshtein Dissimilarity: {levenshtein_diff}")


Levenshtein Dissimilarity: 0.14285714285714285


In [6]:
from scipy.spatial.distance import jensenshannon
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

def jensen_shannon_dissimilarity(distractor, correct_answer):
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform([distractor, correct_answer]).toarray()

    # The Jensen-Shannon Divergence requires probability distributions, not raw frequencies.
    # Normalize each sentence's vector (to sum to 1)
    tfidf_matrix = tfidf_matrix / tfidf_matrix.sum(axis=1)[:, np.newaxis]
    
    # Calculate Jensen-Shannon Divergence
    return jensenshannon(tfidf_matrix[0], tfidf_matrix[1])

# Example sentences
distractor = "Timothy merasa senang dengan beban yang berat."
correct_answer = "Timothy mempunyai beban yang berat."

js_diff = jensen_shannon_dissimilarity(distractor, correct_answer)
print(f"Jensen-Shannon Dissimilarity: {js_diff}")


Jensen-Shannon Dissimilarity: 0.5302047779533421


In [ ]:
def lexical_diversity(distractor: str) ->  float:
    words = distractor.split()
    unique_words = set(words)
    return len(unique_words) / len(words) if words else 0

def 
def length_filter(distractors, correct_answer, max_length_diff=5):
    correct_length = len(correct_answer.split())
    filtered_distractors = [
        d for d in distractors if abs(len(d.split()) - correct_length) <= max_length_diff 
    ]
    return filtered_distractors
list_distractors = [' Timotius mempunyai harapan yang lebih baik.',
 ' Timotius mempunyai banyak waktu.',
 ' Timotius mempunyai harapan yang lebih rendah dibandingkan tepat waktu.',
 '',
 '  Timothy bersemangat dengan harga murah.',
 '  Timothy bersemangat dengan penampilannya yang buruk.',
 '  Timothy bersemangat dengan penampilannya.',
 '',
 ' Tidak ada yang mempunyai banyak waktu.',
 ' Tidak ada yang mempunyai banyak waktu luang dan menonton pertandingan selama begitu cepat.',
 ' Tidak ada yang mempunyai banyak waktu luang.',
 '',
 ' ketika mereka memiliki terlalu banyak waktu luang, Timothy adalah seorang guru yang suka menghabiskan tiga belas jam sehari.',
 ' ketika mereka memiliki terlalu banyak waktu luang, Timothy adalah seorang guru yang suka menghabiskan tiga belas jam sehari di meja kecil.',
 ' ketika mereka memiliki terlalu banyak waktu luang, Timothy adalah seorang guru yang suka mengajar kebahagiaannya.',
 '',
 ' kemampuan mereka menjadi lebih kuat.',
 ' kemampuan mereka menjadi lebih baik.',
 ' kemampuan mereka menjadi lebih berharga.',
 '',
 ' kehidupan yang sulit dipelajari Timothy dan membuatnya bahagia.',
 ' kehidupan yang sulit dipelajari Timothy dan membuatnya lebih bahagia.',
 ' kehidupan yang sulit dipelajari Timothy dan membuatnya merasa lebih bahagia.',
 '']
length_filter(list_distractors, "Timothy mempunyai beban yang berat")

[' Timotius mempunyai harapan yang lebih baik.',
 ' Timotius mempunyai banyak waktu.',
 ' Timotius mempunyai harapan yang lebih rendah dibandingkan tepat waktu.',
 '',
 '  Timothy bersemangat dengan harga murah.',
 '  Timothy bersemangat dengan penampilannya yang buruk.',
 '  Timothy bersemangat dengan penampilannya.',
 '',
 ' Tidak ada yang mempunyai banyak waktu.',
 ' Tidak ada yang mempunyai banyak waktu luang.',
 '',
 '',
 ' kemampuan mereka menjadi lebih kuat.',
 ' kemampuan mereka menjadi lebih baik.',
 ' kemampuan mereka menjadi lebih berharga.',
 '',
 ' kehidupan yang sulit dipelajari Timothy dan membuatnya bahagia.',
 ' kehidupan yang sulit dipelajari Timothy dan membuatnya lebih bahagia.',
 ' kehidupan yang sulit dipelajari Timothy dan membuatnya merasa lebih bahagia.',
 '']

In [ ]:

def clean_dg_1_combine(texts: list) -> str:
    distractors = []
    for tex in texts:
        inner = tex.split("</s>")
        for in_ in inner:
            new_text = re.sub("<[^>]+>", "", in_)
            distractors.append(new_text)
    return distractors

clean_dg_1_combine_data = [clean_dg_1_combine(d) for d in dg_1_val_combine]
clean_dg_1_combine_data[0]

[' Timotius mempunyai harapan yang lebih baik.',
 ' Timotius mempunyai banyak waktu.',
 ' Timotius mempunyai harapan yang lebih rendah dibandingkan tepat waktu.',
 '',
 '  Timothy bersemangat dengan harga murah.',
 '  Timothy bersemangat dengan penampilannya yang buruk.',
 '  Timothy bersemangat dengan penampilannya.',
 '',
 ' Tidak ada yang mempunyai banyak waktu.',
 ' Tidak ada yang mempunyai banyak waktu luang dan menonton pertandingan selama begitu cepat.',
 ' Tidak ada yang mempunyai banyak waktu luang.',
 '',
 ' ketika mereka memiliki terlalu banyak waktu luang, Timothy adalah seorang guru yang suka menghabiskan tiga belas jam sehari.',
 ' ketika mereka memiliki terlalu banyak waktu luang, Timothy adalah seorang guru yang suka menghabiskan tiga belas jam sehari di meja kecil.',
 ' ketika mereka memiliki terlalu banyak waktu luang, Timothy adalah seorang guru yang suka mengajar kebahagiaannya.',
 '',
 ' kemampuan mereka menjadi lebih kuat.',
 ' kemampuan mereka menjadi lebih baik.

In [65]:
def split_dg_1(text: str)-> str:
    return text.split("</s>")

def clean_dg_1(text: str) -> str:
    distractors = []
    texts = split_dg_1(text)
    for tex in texts:
        new_text = re.sub("<[^>]+>", "", tex)
        distractors.append(new_text)
    return distractors

clean_dg_1_data = [clean_dg_1(d) for d in dg_1_val]
ds1, ds2, ds3, ds4, ds4, ds5, ds6, ds7, ds8, ds9, ds10, = [], [], [], [],[], [], [], [], [], [], []
for content in clean_dg_1_data:
    ds1.append(content[0])
    ds2.append(content[1])
    ds3.append(content[2])
    ds4.append(content[3])
    ds5.append(content[4])
    ds6.append(content[5])
    ds7.append(content[6])
    ds8.append(content[7])
    ds9.append(content[8])
    ds10.append(content[9])
val_dg_1["incorrect_1_gen"] = ds1
val_dg_1["incorrect_2_gen"] = ds2
val_dg_1["incorrect_3_gen"] = ds3
#val_dg_1["incorrect_4"] = ds4
#val_dg_1["incorrect_5"] = ds5
#val_dg_1["incorrect_6"] = ds6
#val_dg_1["incorrect_7"] = ds7
#val_dg_1["incorrect_8"] = ds8
#val_dg_1["incorrect_9"] = ds9
#val_dg_1["incorrect_10"] = ds10

In [44]:
dg_1_val[3]

'<pad> kebanyakan orang berpendapat bahwa teknologi tersebut dapat membantu menghasilkan lebih banyak uang.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad> kebanyakan orang berpendapat bahwa teknologi tersebut dapat membantu menghasilkan lebih banyak uang</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad> kebanyakan orang berpendapat bahwa teknologi tersebut dapat membantu menghasilkan lebih banyak uang untuk pekerjaan lainnya.</s><pad> kebanyakan orang berpendapat bahwa teknologi tersebut dapat membantu menghasilkan lebih banyak uang di masa depan</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad> kebanyakan orang berpendapat bahwa teknologi tersebut dapat membantu menghasilkan lebih banyak uang untuk pekerjaan lainnya</s><pad><pad> kebanyakan orang berpendapat bahwa teknologi tersebut dapat membantu menghasilkan lebih banyak uang untuk pekerjaan mereka.</s><pad><pad><pad><pad> kebanyakan orang berpendapat bahwa teknologi tersebut dapat m

# こんとる

In [7]:
#train_tyq_indo.to_csv("tydqa_train.csv")
#val_tyq_indo.to_csv("tydqa_val.csv")

In [ ]:

def clear_qg(text : str) -> str:
    pattern = "<[^>]*>"
    res = re.sub(pattern, "", text)
    return res
clean_qg_pred = [clear_qg(text) for text in qg_val]
print(len(clean_qg_pred))

5928


In [9]:
def clear_qag(text: str) -> str:
    try:
        pattern = "<[^>]+>"
        answer, question = text.split("<question>")
        res_answer = re.sub(pattern, "", answer)
        res_question = re.sub(pattern, "", question)
        return res_answer.strip(), res_question.strip()
    except:
        return "<UNK>", "<UNK>"

qag_answers = []
qag_questions = []

for text in qag_val:
    answer, question = clear_qag(text)
    qag_answers.append(answer)
    qag_questions.append(question)

len(qag_answers), len(qag_questions)
val_qag["answer_gen"] = answer
val_qag["question_gen"] = question

In [10]:
val_qg["question_gen"] = clean_qg_pred

In [11]:
def clear_dg(text: str) -> str:
    clean_distractors = []
    distractors = text.split("<sep>")
    pattern = "<[^>]+>"
    for distractor in distractors:
        clean_text = re.sub(pattern, "", distractor).strip()
        clean_distractors.append(clean_text)
    return clean_distractors

no_token_dg = [clear_dg(text) for text in dg_all_val]
distractor_1 = [text[0] for text in no_token_dg]
distractor_2 = []
distractor_3 = []
for n in no_token_dg:
    try: 
        text = n[1]
        distractor_2.append(text)
    except:
        distractor_2.append("<UNK>")

for n in no_token_dg:
    try:
        text = n[2]
        distractor_3.append(text)
    except:
        distractor_3.append("<UNK>")

print(len(distractor_1))
print(len(distractor_2))
print(len(distractor_3))

4847
4847
4847


In [12]:
distractor_3

['Kesepakatan pada anak-anak itu penting bagi Timur Bahama.',
 '<UNK>',
 '<UNK>',
 'perekonomian Asia Pasifik dan Eropa dapat memberikan manfaat bagi antaranya.',
 'anak-anak terlalu berapahan lingkungan berpengaruh secara langsung',
 '<UNK>',
 'Kebanyakan orang mempunyai kebiasaan buruk',
 '<UNK>',
 'Ada perbedaan antara pengambilan keputusan asing dan sering digantanya',
 'Saat ini, kami selalu memikirkan pemanggilan terkenal saya secara langsung.',
 'Semua anak laki-laki saya sedang mencari pekerjaan',
 'Orang tua saya bisa membayar lebih banyak uang daripada mereka',
 'Saya pergi ke bioskop dengan cara yang benar',
 'Putrinya menghabiskan banyak waktu dengan teman-temannya',
 'Jack membuat orang terkenal di dunia dengan belajar sendiri, tapi mereka perlu tabungannya!',
 'merasa sangat bahagia setelah dia meninggal',
 'Kecepatan tinggi',
 'Saat itu terlalu jauh untuk mengambil keputusan.',
 'Anak perempuan selalu menghabiskan banyak waktu untuk masa depan',
 'hubungan',
 '<UNK>',
 '

In [13]:
def get_len(text: str) -> int:
    return len(text.split(" "))

In [14]:
val_dg["incorrect_1_gen"] = distractor_1
val_dg["incorrect_2_gen"] = distractor_2
val_dg["incorrect_3_gen"] = distractor_3

In [15]:
# Analysis
import numpy as np
def get_automatic_metrics_result(df: pd.DataFrame, column_target: str, column_pred: str):
    bleu_1, bleu_2, bleu_3, bleu_4, rouge_l = [], [], [], [], []
    for indx, item in df.iterrows():
        target = item[column_target]
        pred = item[column_pred]
        b1, b2, b3 ,b4 = AutomaticMetrics()._calculate_bleu(target, pred)
        _, _, _, rouge = AutomaticMetrics()._calculate_rouge(target,pred)

        bleu_1.append(b1)
        bleu_2.append(b2)
        bleu_3.append(b3)
        bleu_4.append(b4)
        rouge_l.append(rouge)
    
    return bleu_1, bleu_2, bleu_3, bleu_4, rouge_l

d1_b1, d1_b2, d1_b3, d1_b4, d1_rouge = get_automatic_metrics_result(val_dg, "incorrect_1", "incorrect_1_gen")
d2_b1, d2_b2, d2_b3, d2_b4, d2_rouge = get_automatic_metrics_result(val_dg, "incorrect_2", "incorrect_2_gen")
d3_b1, d3_b2, d3_b3, d3_b4, d3_rouge = get_automatic_metrics_result(val_dg, "incorrect_3", "incorrect_3_gen")
        

/mnt/Dev_partition/anaconda3/envs/e2e_indonesian_amcq_qg/lib/python3.9/site-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/mnt/Dev_partition/anaconda3/envs/e2e_indonesian_amcq_qg/lib/python3.9/site-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/mnt/Dev_partition/anaconda3/envs/e2e_indonesian_amcq_qg/lib/python3.9/site-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently

In [66]:
d1_1_b1, d1_1_b2, d1_1_b3, d1_1_b4, d1_1_rouge = get_automatic_metrics_result(val_dg_1, "incorrect_1", "incorrect_1_gen")
d2_1_b1, d2_1_b2, d2_1_b3, d2_1_b4, d2_1_rouge = get_automatic_metrics_result(val_dg_1, "incorrect_2", "incorrect_2_gen")
d3_1_b1, d3_1_b2, d3_1_b3, d3_1_b4, d3_1_rouge = get_automatic_metrics_result(val_dg_1, "incorrect_3", "incorrect_3_gen")

/mnt/Dev_partition/anaconda3/envs/e2e_indonesian_amcq_qg/lib/python3.9/site-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/mnt/Dev_partition/anaconda3/envs/e2e_indonesian_amcq_qg/lib/python3.9/site-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/mnt/Dev_partition/anaconda3/envs/e2e_indonesian_amcq_qg/lib/python3.9/site-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently

In [16]:
def print_mean(bleu_d_1, bleu_d_2, bleu_d_3, bleu_d_4, rouge_l):
    print("Bleu 1 : {}".format(np.mean(bleu_d_1)))
    print("Bleu 2 : {}".format(np.mean(bleu_d_2)))
    print("Bleu 3 : {}".format(np.mean(bleu_d_3)))
    print("Bleu 4 : {}".format(np.mean(bleu_d_4)))
    print("rouge l : {}".format(np.mean(rouge_l)))
print("Distractor : 1")
print_mean(d1_b1, d1_b2, d1_b3, d1_b4, d1_rouge)

print("Distractor : 2")
print_mean(d2_b1, d2_b2, d2_b3, d2_b4, d2_rouge)

print("Distractor : 3")
print_mean(d3_b1, d3_b2, d3_b3, d3_b4, d3_rouge)

Distractor : 1
Bleu 1 : 0.4710519162861399
Bleu 2 : 0.3272908211092453
Bleu 3 : 0.22739299110342237
Bleu 4 : 0.16095068893914014
rouge l : 0.12018443143027759
Distractor : 2
Bleu 1 : 0.3475355197597709
Bleu 2 : 0.2437731942161575
Bleu 3 : 0.1610754021671333
Bleu 4 : 0.10033167558157058
rouge l : 0.05816075136410953
Distractor : 3
Bleu 1 : 0.33107628687883905
Bleu 2 : 0.20773937077629426
Bleu 3 : 0.12271373860865316
Bleu 4 : 0.06569363584858638
rouge l : 0.030276881338422035


In [67]:
print("DISTRACTOR_1_RESULT")
print("Distractor : 1")
print_mean(d1_1_b1, d1_1_b2, d1_1_b3, d1_1_b4, d1_1_rouge)

print("Distractor : 2")
print_mean(d2_1_b1, d2_1_b2, d2_1_b3, d2_1_b4, d2_1_rouge)

print("Distractor : 3")
print_mean(d3_1_b1, d3_1_b2, d3_1_b3, d3_1_b4, d3_1_rouge)

DISTRACTOR_1_RESULT
Distractor : 1
Bleu 1 : 0.44042530018780296
Bleu 2 : 0.31648648351259684
Bleu 3 : 0.2247250311222606
Bleu 4 : 0.16162008367009975
rouge l : 0.12215555378793186
Distractor : 2
Bleu 1 : 0.4501747608604313
Bleu 2 : 0.32092521259686496
Bleu 3 : 0.22424322038549255
Bleu 4 : 0.15829597698787784
rouge l : 0.11338339245907922
Distractor : 3
Bleu 1 : 0.4543828248439375
Bleu 2 : 0.3237833168462842
Bleu 3 : 0.22584177063602556
Bleu 4 : 0.15975019660128215
rouge l : 0.11310783907517147


In [17]:
qg_b1, qg_b2, qg_b3, qg_b4, qg_rouge = get_automatic_metrics_result(val_qg, "question", "question_gen")
print_mean(qg_b1, qg_b2, qg_b3, qg_b4, qg_rouge)

Bleu 1 : 0.5978142529745514
Bleu 2 : 0.5013427214112683
Bleu 3 : 0.4322662270016423
Bleu 4 : 0.38098791678883304
rouge l : 0.3720007482150228


In [18]:
from automatic_metrics import AutomaticMetrics
bleu_qg_1, bleu_qg_2, bleu_qg_3, bleu_qg_4, rouge_l = [], [], [], [], []
for indx, item in val_qg.iterrows():
    target = item["question"]
    pred = item["question_gen"]
    b1, b2, b3, b4 = AutomaticMetrics()._calculate_bleu(target, pred)
    _, _, _, r = AutomaticMetrics()._calculate_rouge(target, pred)

    bleu_qg_1.append(b1)
    bleu_qg_2.append(b2)
    bleu_qg_3.append(b3)
    bleu_qg_4.append(b4)
    rouge_l.append(r)

In [19]:
print(np.mean(bleu_qg_1))
print(np.mean(bleu_qg_2))
print(np.mean(bleu_qg_3))
print(np.mean(bleu_qg_4))
print(np.mean(rouge_l))

0.5978142529745514
0.5013427214112683
0.4322662270016423
0.38098791678883304
0.3720007482150228


In [20]:
val_qg.to_csv("generated/qg_for_analysis.csv", index=False)
val_dg.to_csv("generated/dg_all_for_analysis.csv", index=False)
val_qag.to_csv("generated/qag_for_analysis.csv", index=False)

In [21]:
n=100
val_qg.sample(n).to_csv("generated/qg_for_analysis_{}.csv".format(n), index=False)
val_dg.sample(n).to_csv("generated/dg_all_for_analysis_{}.csv".format(n), index=False)
val_qag.sample(n).to_csv("generated/qag_for_analysis_{}.csv".format(n), index=False)